# 0. Initialize

## 0.1. Import Libraries

In [8]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

from IPython import display
import matplotlib as mpl
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0.2. DEFINE VARIABLES 

In [9]:
DATA_PATH = '/Users/onurvarol/Downloads/data/' # '<insert-your-training-data-path-here>'

ROUND = 3 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID = '28195'#'<insert-your-id-here>'
PROJECT_CODE = 'CS412da78f316bc56'#'<insert-your-code-here>' # Same code for the annotation eg. CS412xxxxx

In [10]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH ='/content/drive/My Drive/CS412_project/raw_data/'

# trainingTweetDf = pd.read_csv('/content/drive/My Drive/cs412_project/annotated_tweets_CS412da78f316bc56.csv',dtype={'tweet_id': str, 'isPolitical': str})
# trainingUserDf = pd.read_csv('/content/drive/My Drive/cs412_project/annotated_users_CS412da78f316bc56.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 0.3. Read Training & Evaluation Data

### 0.3.1. Get the labels for tweets

In [11]:
#trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH))
trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})
trainingTweetDf

,tweet_id,isPolitical
0,1597170281545551872,Yes
1,1431700027471192069,No
2,1566035577090281472,Yes
3,1591538690869940225,Yes
4,1583898169238167554,Yes
...,...,...
2995,1593539327623151619,Yes
2996,1393886554062524418,No
2997,1597925615092764672,Yes
2998,1585291418616176640,Yes


In [12]:
trainingTweetDf.isPolitical.value_counts()

Yes    2003
No      997
Name: isPolitical, dtype: int64

### 0.3.2. Get the labels for users

In [13]:
trainingUserDf = pd.read_csv('{}training-user.csv'.format(DATA_PATH))
#trainingUserDf = pd.read_csv('training-user.csv')
trainingUserDf

,screen_name,isBot
0,koftecancaddy,No
1,ahaber,No
2,selahat03949652,No
3,erdin06357062,No
4,bhct__necatii,No
...,...,...
2995,djblumenberg,No
2996,mel1sq,No
2997,eren_yz1,Yes
2998,ergnyildiz4,No


In [14]:
trainingUserDf.isBot.value_counts()

No     2424
Yes     576
Name: isBot, dtype: int64

### 0.3.3. Expand your dataset with metadata and tweets

In [15]:
# You can also expand training data by downloading your own labeled datasets following the link
# Download the documents under "Link to training data"

print('http://www.onurvarol.com/Annotation-CS412-202201/reports/report_{}.html'.format(PROJECT_CODE))

http://www.onurvarol.com/Annotation-CS412-202201/reports/report_CS412da78f316bc56.html


In [18]:
trainingTweetDf_myData = pd.read_csv('/content/drive/My Drive/CS412_project/annotated_tweets_CS412da78f316bc56.csv',dtype={'tweet_id': str, 'isPolitical': str})
trainingUserDf_myData = pd.read_csv('/content/drive/My Drive/CS412_project/annotated_users_CS412da78f316bc56.csv')

In [19]:
trainingTweetDf_myData.head()

,Unnamed: 0,url,sentiment,isExperiential,isPolitical,isInsult,topics
0,1561055903297806342,https://docs.google.com/forms/d/e/1FAIpQLSffZu...,Negative,No,Yes,Yes,"['News', 'Democracy', 'Election']"
1,1596907864005697539,https://docs.google.com/forms/d/e/1FAIpQLSffZu...,Negative,Yes,Yes,No,"['Economy', 'Inflation&poverty', 'Unemployment..."
2,1579558096833511424,https://docs.google.com/forms/d/e/1FAIpQLSffZu...,Negative,Yes,Yes,No,"['News', 'Education', 'Democracy', 'Crime&just..."
3,1596946571706634240,https://docs.google.com/forms/d/e/1FAIpQLSffZu...,Negative,Yes,Yes,Yes,"['Democracy', 'Election', 'Crime&justice']"
4,1436963996008079363,https://docs.google.com/forms/d/e/1FAIpQLSffZu...,Neutral,Yes,No,Yes,['Sports']


# 1. EXTRACT FEATURES
Under *1.1. Political Tweet Detection* and *1.2. Bot Detection*, we firstly collect raw data for processing. We then combine some of them (total_interactions = num_favorites + num_retweets) or use them to extract features (whether the tweet has one of the political entities @meralaksener, @kilicdarogluk etc.).

We expect you to collect more raw data from **tweet_metadata**, **user_profiles** and **user_tweets** files by creating a function as shown in below examples such as *check_if_retweet()* and using it while iterating over data as shown under *Merge Collected Features*.

We also expect you to create new variables as much as you can from the data in order to make your predictions more accurate. For example, you may want to check:

- The tweet sources that a user frequently uses
- Whether the user is a verified account or not

...

to assess whether **a user is a bot or not** and whether **a tweet is political or not**.

In [20]:
PATH_TO_DOWNLOADED = DATA_PATH # 'D:/Users/suuser/Desktop/Sabancı/CS412/spring-2022/project/'

## 1.1. Political Tweet Detection
This part stands for the feature extraction of tweets. We start with collecting the raw data from *tweet_metadata*, then use some of them to extract features.

### 1.1.1. Get Raw Data

#### 1.1.1.1. Check if Retweet

In [21]:
def check_if_retweet(tweet_metadata_line):
    is_retweet = 0
    retweeted_username = None

    try:
        tweet_metadata_line['retweeted_status']
        retweeted_username = tweet_metadata_line['retweeted_status']['user']['screen_name'].lower()
        is_retweet = 1

    except KeyError:
        pass

    return is_retweet, retweeted_username

#### 1.1.1.2. Get Tweet Text

In [22]:
def get_tweet_text(tweet_metadata_line):
    text = tweet_metadata_line['text']
    
    return text

#### 1.1.1.3. Get Tweet ID

In [23]:
def get_tweet_id(tweet_metadata_line):
    id_str = tweet_metadata_line['id_str']
    
    return id_str

#### 1.1.1.4. Get Number of Mentions and Hashtags

In [24]:
def get_number_mentions_hashtags(tweet_metadata_line):
    num_mentions = len(tweet_metadata_line['entities']['user_mentions'])
    num_hashtags = len(tweet_metadata_line['entities']['hashtags'])
    num_url = len(tweet_metadata_line['entities']['urls'])
   

    return num_mentions, num_hashtags,num_url

In [25]:
def check_if_political_mentions(tweet_metadata_line):
    
    mention = tweet_metadata_line['entities']['user_mentions']
    
    political_people = ['M_Sarigul', 'OPAMUKOGLU', 'RT_Erdogan', 'safakpavey', 'cbabdullahgul',
    '06melihgokcek', 'EgemenBagis', 'kilicdaroglu', 'suatkilic', 'memetsimsek',
    'bulent_arinc', 'vekilince', 'dbdevletbahceli', 'samiltayyar27', 'NumanKurtulmus',
    'Notredamedesion', 'sirsureyya', 'BalbayMustafa', 'gurseltekin34', 'bdpdemirtas',
    'UfukUras', 'Nimet_Bas', 'sinan_ogan', 'Av_SelamiOzturk', 'saffetsancakli', 
    'LutfuTurkkan', 'edibesozen', 'emin_ekmen', 'aktaserdogan', 'tunabeklevic',
    'Rasimozankut', 'kamalakmustafa', 'Kader_Sevinc', 'EsenErmis', 'feyziisbasaran', 
    'Ahmet_Davutoglu', 'emrehanhalici', 'aykuterdogdu', 'hilal9can', 'AylinNazliaka', 
    'MSTanrikulu', 'VeyselEroglu', 'tuncelsebahat', 'Kadir__Topbas', 'Valimutlu', 'BakanGuler', 
    'alp_kavaklioglu', 'ABabuscu', 'zkarahanuslu', 'AyhanSeferUSTUN', 'Salih_Kapusuz', 'mustafaelitas', 
    'aaydin02', 'salimuslu_', 'huseyinburge', 'burhankuzu', 'BilalMacit', 'halukipek06', 'guldalaksit', 
    'MustafaSentop', 'erginsadullah', 'yigitbulutt', 'Hayati_Yazici', 'bybekirbozdag', 'MuezzinogluDR', 
    'FatmaSahin_ASPB', 'omerrcelik', 'ertanaydin', 'abdulkadiraksu1', 'csgbfarukcelik', 'oktayvural', 
    'erdemekrem', 'hc_huseyincelik', 'suleymansoylu', 'cevdetyilmaz', 'ErdoganBayraktr', 'Y_Akdogan', 
    'AbdurrahmanBsk', 'oguzksalici', 'drerhanerol', 'hakansukur', 'Mustafa_Destici', 'munirkaraloglu', 
    'AhmettPoyraz', 'Fazli_Kilic', 'MustafaYMN', 'buyuk_mustafa', 'DuzgunOrhan', 'ibrahimozcimen', 
    'drsadiyazici', 'AHASIMIBALTACI', 'baydemirosman', 'SalihKenan', 'osmandevelioglu', 'CAltunay', 
    'n_kadioglu', 'OsmanBoyraz', 'mustafakara', 'murataydintr', 'Mustafa___Demir', 'Hasan_Cann', 
    'KemalBurkay1', 'cumaicten', 'MevlutCavusoglu', 'erolkaya_ist', 'SIRIN_UNAL', 'y_celikbilek', 
    'yeniay_aziz', 'mevlutuysal_bsk', 'sakiksirri', 'KorayaydinMHP', 'AtilaKaya_MHP', 'OktayOzturk_MHP', 
    'MustafaERDEMMHP', 'yusufhalacoglu']
    if( mention != []):
      result = filter(lambda x: x["screen_name"] in political_people , mention)
      length_result = len(list(result));
      return length_result
   
    return 0


#### 1.1.1.5. Get Number of Retweets and Favorites

In [26]:
def get_number_retweets_favorites(tweet_metadata_line):
    retweet_count = tweet_metadata_line['retweet_count']
    favorite_count = tweet_metadata_line['favorite_count']
    
    return retweet_count, favorite_count

#### 1.1.1.6. Get User Info

In [27]:
def get_user_info(tweet_metadata_line):
    id = tweet_metadata_line['user']['id_str']
    screen_name = tweet_metadata_line['user']['screen_name'].lower()
    description = tweet_metadata_line['user']['description']
    verified=tweet_metadata_line['user']['verified'] #added

    return id, screen_name, description,verified

In [28]:
def get_tweet_lang(tweet_metadata_line):
    lang_tweet = tweet_metadata_line['lang']
    #lang_user=tweet_metadata_line['user']['lang']
    # print("-----")
    # print(lang_tweet)
    # print(lang_user)

    return lang_tweet

### 1.1.2. Derive Manually Crafted Features

#### 1.1.2.1. Check for political entity in text

In [29]:
def check_political_ent(text):
    
    # the list below can be modified and some new names may be added (or removed)
    list_of_entities = ['meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag', 'erdogan','receptayyiperdogan'
                        ,'kemalkilicdaroglu', 'mansur_yavas', 'mansuryavas', 'imamoglu', 'ekremimamoglu', 'ekrem_imamoglu', 'umit_ozdag', 'devlet bahceli',
                        'devletbahceli', 'meral akşener', 'davutoglu', 'ahmet_davutoglu', 'eczozgurozel', 'ozgurozel', 'dogu_perincek', 'hdpdemirtas',
                        'T_Karamollaoglu', 'meral_aksener']
    
    entities_in_text = [ent for ent in list_of_entities if ent.lower() in text.lower()]
    number_entities = len(entities_in_text)

    return number_entities

In [30]:
def check_political_words(text):
    
    # the list below can be modified and some new names may be added (or removed)
    list_of_words = ['lider', 'CHP', 'mülteci', 'chp', 'terörist', 'selahattin demirtaş', 'ali babacan', 'başkanım', 'demokrasi',
                     'yasama', 'kemal kılıçdaroğlu', 'mansur yavaş', 'kurul', 'zam', 'belediye', 'rejim', 'ümit özdağ',
                     'parti', 'recep tayyip erdoğan', 'fetöcü', 'iyi_parti', 'parlamento', 'gençlik kollar',
                     'devlet bahceli', 'mhp', 'cumhur ittifakı', 'hdp', 'teror', 'silivri', 'yargi', 'erdogan',
                     'bakan', 'başkan', 'iyi parti', 'özgürlük', 'oyumuz', 'askeri', 'siyaset', 'sosyal adalet',
                     'siyasi', 'bakanım', 'genel başkan', 'af', 'döviz', 'yasama organı', 'devlet', 'dolar', 'DEVA',
                     'ekrem imamoğlu', 'protesto', 'suriyeli', 'karar', 'sosyal politika', 'terör', 'zillet ittifakı',
                     'danıştay', 'dava', 'ideoloji', 'fetö', 'uzun adam', 'millet ittifakı', 'yerel seçim', 'meral akşener',
                     'seçim', 'siyasal', 'ozgurluk', 'tayyip', 'HDP', 'afgan', 'devlet yönetimi', 'mahkeme', 'aday',
                     'siyasi lider', 'cezaevi', 'meclis', 'adalet', 'İyi Parti', 'erken seçim', 'MHP', 'ahmet davutoğlu',
                     'muhalefet', 'darbeci', 'erken secim', 'anayasa', 'yargı organı', 'milletvekil', 'yargı',
                     'cumhurbaskani', 'oy', 'altılı masa', 'yürütme organı', 'mehmetçik', 'süleyman soylu', 'kurultay',
                     'ak parti', 'din', '15 temmuz', 'sedat peker', 'darbe', 'kadın kolları', 'enflasyon', 'sağcı',
                     'miting', 'cumhurbaşkanı', 'politika', 'hukuk', 'euro', 'binali yıldırım', 'genel seçim',
                     'yasalar', 'milletvekili', 'bütçe', 'feto', 'devlet bahçeli', 'tbmm', 'vergi', 'asker', 'butce',
                     'ekonomi', 'AKP', 'secim', 'genel baskan', 'türkiye', 'güvenlik', 'iktidar', 'solcu', 'yandaş',
                     'reis', 'ülkücü', 'seçmen', 'reisim', 'vatandaş', 'erdoğan', 'milliyetçilik', 'hükümet', 'pkk']
    
    words_in_text = [word for word in list_of_words if word.lower() in text.lower()]
    number_pol_words = len(words_in_text)

    return number_pol_words

In [31]:
def get_text_len(text):
    
    text_len = len(text)

    return text_len

#### 1.1.2.2. Number of total interactions

In [32]:
def total_interactions(retweet_count, favorite_count):
    total_num_interactions = retweet_count + favorite_count
    
    return total_num_interactions

In [33]:
def get_retweet_interactions_ratio(retweet_count, favorite_count):
    total_num_interactions = retweet_count + favorite_count

    if total_num_interactions == 0:
      retweet_ratio = 0
    else:
        retweet_ratio= retweet_count/ (total_num_interactions)
    
    return retweet_ratio

In [34]:
def count_digits(line):
    count = 0
    for i in line:
        if i.isdigit():
            return True
    return False

#### 1.1.2.3. Get uppercase / length of the tweet ration

In [35]:
import re
def get_uppercase_ratio(text):
    uppercase = re.findall('[A-Z]',text)
    ratio = len(uppercase) / len(text)
    return ratio

In [36]:

dfPolitical = {'tweet_id':[],
              'is_retweet':[],
              'retweeted_username':[],
              'text':[],
              'num_mentions':[],
              'num_hashtags':[],
              'num_url':[],
              'num_retweets':[],
              'num_favorites':[],
              'user_id':[],
              'user_screen_name':[],
              'user_description':[],
              'num_political_entities':[],
              'total_interactions':[],
               
              'lang_tweet':[],
              'user_verified':[],
              'num_political_words':[],
              'retweet_interactions_ratio':[],
              'text_len':[],
              'uppercase_ratio': [],
              'num_of_political_mentions': [],
              'digit_exists': []}


with gzip.open(f"{PATH_TO_DOWNLOADED}tweet_metadata.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)
     
        # raw data:
        id_str = get_tweet_id(line)
        is_retweet, retweeted_username = check_if_retweet(line)
        text = get_tweet_text(line)
        num_mentions, num_hashtags, num_url = get_number_mentions_hashtags(line)
        
        retweet_count, favorite_count = get_number_retweets_favorites(line)
        user_id_str, screen_name, user_description,user_verified = get_user_info(line)
        tweet_lang =get_tweet_lang(line)

        # manually crafted data:
        num_political_entities = check_political_ent(text)
        total_num_interactions = total_interactions(retweet_count, favorite_count)

        num_political_words = check_political_words(text)
        retweet_interactions_ratio=  get_retweet_interactions_ratio(retweet_count, favorite_count)
        text_len=  get_text_len(text)
        
        uppercase_ratio = get_uppercase_ratio(text)

        num_of_political_mentions = check_if_political_mentions(line)

        digit_exists = count_digits(screen_name)


        dfPolitical['tweet_id'].append(id_str)
        dfPolitical['is_retweet'].append(is_retweet)
        dfPolitical['retweeted_username'].append(retweeted_username)
        dfPolitical['text'].append(text)
        dfPolitical['num_mentions'].append(num_mentions)
        dfPolitical['num_hashtags'].append(num_hashtags)
        dfPolitical['num_url'].append(num_url)
        dfPolitical['num_retweets'].append(retweet_count)
        dfPolitical['num_favorites'].append(favorite_count)
        dfPolitical['user_id'].append(user_id_str)
        dfPolitical['user_screen_name'].append(screen_name)
        dfPolitical['user_description'].append(user_description)
        dfPolitical['num_political_entities'].append(num_political_entities)
        dfPolitical['total_interactions'].append(total_num_interactions)

        dfPolitical['lang_tweet'].append(tweet_lang)
        dfPolitical['user_verified'].append(user_verified)
        dfPolitical['num_political_words'].append(num_political_words)
        dfPolitical['retweet_interactions_ratio'].append(retweet_interactions_ratio)
        dfPolitical['text_len'].append(text_len)

        dfPolitical['uppercase_ratio'].append(uppercase_ratio)
        dfPolitical['num_of_political_mentions'].append(num_of_political_mentions)

        dfPolitical['digit_exists'].append(digit_exists)
        
      

In [37]:
dfPolitical = pd.DataFrame(dfPolitical)
dfPolitical

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_url,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,lang_tweet,user_verified,num_political_words,retweet_interactions_ratio,text_len,uppercase_ratio,num_of_political_mentions,digit_exists
0,1588568792984346624,0,None,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,1,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,0,147,tr,False,0,0.333333,140,0.035714,0,False
1,1588452263047069697,0,None,"@mahirunal Gavur İzmir ya onlar, hani Cumhuriy...",1,0,0,0,0,595514060,mtfdan,,0,0,tr,False,0,0.000000,97,0.020619,0,False
2,1569589330544398336,0,None,#ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...,0,1,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,0,0,tr,False,1,0.000000,140,0.150000,0,False
3,1570428119609139201,0,None,@ajans_muhbir Siz kaypak olmayıp onay vermesey...,1,0,1,0,0,1478775431008595968,hamitelkelle,HighOne,0,0,tr,False,1,0.000000,140,0.057143,0,False
4,1551163840368414722,0,None,Engelli öğretmenler olarak önümüzdeki engeller...,0,0,1,0,0,1511976696337113088,sed58417690,,0,0,tr,False,0,0.000000,140,0.042857,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33528,1568595408233832448,0,None,Gerçek kimlik taşımayan hesaplara cevap vermem...,0,0,1,9,81,576247173,ardanzenturk,RT ONAYLADIĞIM ANLAMINA GELMEZ\nArtık fikirler...,0,90,tr,False,0,0.100000,140,0.042857,0,False
33529,1584027427696959488,0,None,@umitozdag Neden Suriyelilerle ilgili bu kadar...,1,0,1,1,8,162308585,ozgul_61,Bridge design engineer Yaay hesabı : dilfiruz,1,9,tr,False,3,0.111111,140,0.064286,0,True
33530,1585945783307730945,0,None,@celebimehmeta Niye Türkiye yüzyılıda.Türkiye ...,1,0,0,0,1,415025519,ladrekova,,0,1,tr,False,1,0.000000,76,0.052632,0,False
33531,1569748909521801221,1,muazzezeralp,RT @muazzezeralp: @Doan58213655 @denizkonur @N...,7,1,0,6,0,1442125177727307781,yapikytgrivrlsn,,2,6,tr,False,2,1.000000,140,0.092857,1,False


## 1.2. From Users

### 1.2.1. Get user metadata from user_profiles.jsons.gz

#### 1.2.1.1. Get user info metadata

In [38]:
def get_user_info_metadata(user_metadata_line):

    
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']

    user_verified = user_metadata_line['verified'] #added
    user_protected = user_metadata_line['protected'] #added
    user_favorites_count=user_metadata_line['favourites_count'] #added
    user_status_count =user_metadata_line['statuses_count'] #added
    user_account_created= user_metadata_line['created_at']  #added
    user_default_profile = user_metadata_line['default_profile_image']
    created_at = user_metadata_line['created_at']

    dictionary = {'user_id':user_id, 'user_name': user_name, 'user_screen_name':user_screen_name, 'user_location':user_location,
     'user_description':user_description, 'user_followers_count':user_followers_count, 'user_friends_count':user_friends_count,
     'user_verified':user_verified,'user_protected':user_protected,'user_favorites_count':user_favorites_count,
     'user_status_count':user_status_count,'user_account_created':user_account_created,'user_default_profile':user_default_profile,'created_at' : created_at}

    return dictionary

#### 1.2.1.2. Get followers/(followers+friends) ratio

In [39]:
def get_followers_all_ratio(user_followers_count, user_friends_count):
    
    if user_friends_count + user_followers_count == 0:
        followers_all_ratio = 0

    else:
        followers_all_ratio =  user_followers_count / (user_friends_count + user_followers_count)

    return followers_all_ratio

In [40]:
!pip install pytz


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from datetime import datetime
import pytz
def get_status_day_ratio(user_account_created, user_status_count):
    
    date_object = datetime.strptime(user_account_created, "%a %b %d %H:%M:%S %z %Y")
    date_object = date_object.date()
    now = datetime.now().date()
    diff= now - date_object

    status_day_ratio= user_status_count/(diff.days)



    return status_day_ratio

In [42]:
from datetime import datetime
import pytz
def get_fav_day_ratio(user_account_created, favourites_count):
    
    date_object = datetime.strptime(user_account_created, "%a %b %d %H:%M:%S %z %Y")
    date_object = date_object.date()
    now = datetime.now().date()
    diff= now - date_object

    fav_day_ratio= favourites_count/(diff.days)



    return fav_day_ratio

#### 1.2.1.3. Get description length

In [43]:
def get_desc_len(user_description):
    
    description_len = len(user_description)

    return description_len

In [44]:
def get_time_difference(created_at):
    date_object = datetime.strptime(created_at, "%a %b %d %H:%M:%S %z %Y")
    formatted_date = date_object.strftime("%d %b %Y")

    date_object = datetime.strptime(formatted_date, "%d %b %Y")
    today = datetime.now()
   
    difference = (today - date_object).days
    return difference


In [45]:
dfBot = {'user_id':[],
         'user_name':[],
         'user_screen_name':[],
         'user_location':[],

         'user_verified':[],
         'user_protected':[],
         'user_favorites_count':[],
         'user_status_count':[],
         'user_account_created':[],
        'user_default_profile':[],
         

         'user_description':[],
         'user_followers_count':[],
         'user_friends_count':[],

         'description_len':[],
         'followers_to_all_ratio':[],

         'status_day_ratio':[],
         'fav_day_ratio':[],

         'created_at' : [],
         'difference_in_days_after_created' : [],
         }


with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)

        
        # manually crafted data:
        description_len = get_desc_len(dictionary['user_description'])
        dfBot['description_len'].append(description_len)
        
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'], 
                                                      dictionary['user_friends_count'])
       

        dfBot['followers_to_all_ratio'].append(followers_all_ratio)

        status_day_ratio = get_status_day_ratio(dictionary['user_account_created'], 
                                                      dictionary['user_status_count'])
        dfBot['status_day_ratio'].append(status_day_ratio)

        fav_day_ratio = get_fav_day_ratio(dictionary['user_account_created'], 
                                                      dictionary['user_favorites_count'])
        dfBot['fav_day_ratio'].append(fav_day_ratio)
        date_difference_until_created = get_time_difference(dictionary['created_at'])
        dfBot['difference_in_days_after_created'].append(date_difference_until_created )

In [46]:
dfBot = pd.DataFrame(dfBot)
dfBot

,user_id,user_name,user_screen_name,user_location,user_verified,user_protected,user_favorites_count,user_status_count,user_account_created,user_default_profile,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,status_day_ratio,fav_day_ratio,created_at,difference_in_days_after_created
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",False,False,17676,2551,Fri Aug 27 13:07:30 +0000 2021,False,Student,65,185,7,0.260000,4.953398,34.322330,Fri Aug 27 13:07:30 +0000 2021,515
1,1304340303080386560,fania :((((,scorpiehoez,bogor,False,False,15474,42771,Fri Sep 11 08:45:44 +0000 2020,False,have a holly jolly🎄,8235,3011,19,0.732260,49.446243,17.889017,Fri Sep 11 08:45:44 +0000 2020,865
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,False,False,18220,14300,Wed Apr 10 18:15:31 +0000 2019,False,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,10.324910,13.155235,Wed Apr 10 18:15:31 +0000 2019,1385
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,False,False,26999,21303,Fri Jan 29 11:01:25 +0000 2016,False,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,8.347571,10.579545,Fri Jan 29 11:01:25 +0000 2016,2552
4,2225373636,SLMDMR,biologselim,,False,False,2179,1629,Sun Dec 01 18:16:41 +0000 2013,False,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,0.487579,0.652200,Sun Dec 01 18:16:41 +0000 2013,3341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,False,False,10820,2396,Mon Oct 26 21:08:22 +0000 2020,False,,229,217,0,0.513453,2.921951,13.195122,Mon Oct 26 21:08:22 +0000 2020,820
29666,111074128,Melda Onur,meldaonur,,False,False,36671,75178,Wed Feb 03 18:39:01 +0000 2010,False,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088,15.867033,7.739764,Wed Feb 03 18:39:01 +0000 2010,4738
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",False,False,7389,6482,Fri May 01 13:56:23 +0000 2009,False,hayat oyunsa bende jeton çok,116,141,28,0.451362,1.292265,1.473086,Fri May 01 13:56:23 +0000 2009,5016
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",False,False,140095,121113,Fri Mar 14 18:05:09 +0000 2014,False,Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431,37.403644,43.265905,Fri Mar 14 18:05:09 +0000 2014,3238


### 1.2.2. Get Tweet Info of Users in user_profiles.jsons.gz

#### 1.2.2.1. Check ratio of retweets to all tweets

In [47]:
def get_retweet_tweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    if total_tweets == 0:
        retweet_total_ratio = None
    else:
        retweet_total_ratio = number_retweets/(total_tweets)
    
    return retweet_total_ratio

#### 1.2.2.2. Check median number of favorites

In [48]:
def get_median_number_favorites(line):
    num_median_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return num_median_favorites

### 1.2.3. Collect data using the functions above and transform into a Pandas DataFrame

In [49]:
dfBotTweets = {'user_id':[],
               'retweet_total_ratio':[],
               'num_median_favorites':[],
               'num_of_tweets':[]
              }

i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    for line in f:

        line = json.loads(line)

        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)
        
        retweet_total_ratio = get_retweet_tweet_ratio(line)
        dfBotTweets['retweet_total_ratio'].append(retweet_total_ratio)
        
        num_median_favorites = get_median_number_favorites(line)
        dfBotTweets['num_median_favorites'].append(num_median_favorites)
        
        dfBotTweets['num_of_tweets'].append(len(line['tweets']))

        i += 1
        if i % 1000 == 0:
            print(i)

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [52]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets

,user_id,retweet_total_ratio,num_median_favorites,num_of_tweets
0,594642154,0.115000,2.0,200
1,525600289,0.005025,1.0,199
2,931895965501534209,0.900000,0.0,200
3,1591543462746329088,0.185000,0.0,200
4,734801354749796352,1.000000,0.0,200
...,...,...,...,...
28310,1591370361488252928,0.800000,0.0,200
28311,1475272459616235525,0.825000,0.0,200
28312,1096753792731750401,0.051020,1.0,196
28313,1269527617687953409,0.095000,2.0,200


### 1.2.3. Merge dfBot and dfBotTweets

In [51]:
dfBotAll = dfBot.merge(dfBotTweets,
                       how='left')

dfBotAll[['retweet_total_ratio', 'num_median_favorites', 'difference_in_days_after_created', 'num_of_tweets']] = dfBotAll[['retweet_total_ratio', 'num_median_favorites', 'difference_in_days_after_created', 'num_of_tweets']].fillna(0)

dfBotAll['difference_in_days_after_created'] = round(dfBotAll['num_of_tweets'] / dfBotAll['difference_in_days_after_created'] ,4)

dfBotAll

,user_id,user_name,user_screen_name,user_location,user_verified,user_protected,user_favorites_count,user_status_count,user_account_created,user_default_profile,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,status_day_ratio,fav_day_ratio,created_at,difference_in_days_after_created,retweet_total_ratio,num_median_favorites,num_of_tweets
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",False,False,17676,2551,Fri Aug 27 13:07:30 +0000 2021,False,Student,65,185,7,0.260000,4.953398,34.322330,Fri Aug 27 13:07:30 +0000 2021,0.3825,0.395939,0.0,197.0
1,1304340303080386560,fania :((((,scorpiehoez,bogor,False,False,15474,42771,Fri Sep 11 08:45:44 +0000 2020,False,have a holly jolly🎄,8235,3011,19,0.732260,49.446243,17.889017,Fri Sep 11 08:45:44 +0000 2020,0.2312,0.125000,0.0,200.0
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,False,False,18220,14300,Wed Apr 10 18:15:31 +0000 2019,False,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,10.324910,13.155235,Wed Apr 10 18:15:31 +0000 2019,0.1444,0.910000,0.0,200.0
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,False,False,26999,21303,Fri Jan 29 11:01:25 +0000 2016,False,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,8.347571,10.579545,Fri Jan 29 11:01:25 +0000 2016,0.0768,0.015306,1.0,196.0
4,2225373636,SLMDMR,biologselim,,False,False,2179,1629,Sun Dec 01 18:16:41 +0000 2013,False,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,0.487579,0.652200,Sun Dec 01 18:16:41 +0000 2013,0.0590,0.659898,0.0,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,False,False,10820,2396,Mon Oct 26 21:08:22 +0000 2020,False,,229,217,0,0.513453,2.921951,13.195122,Mon Oct 26 21:08:22 +0000 2020,0.2439,0.015000,1.0,200.0
29666,111074128,Melda Onur,meldaonur,,False,False,36671,75178,Wed Feb 03 18:39:01 +0000 2010,False,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088,15.867033,7.739764,Wed Feb 03 18:39:01 +0000 2010,0.0420,0.291457,2.0,199.0
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",False,False,7389,6482,Fri May 01 13:56:23 +0000 2009,False,hayat oyunsa bende jeton çok,116,141,28,0.451362,1.292265,1.473086,Fri May 01 13:56:23 +0000 2009,0.0389,0.061538,0.0,195.0
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",False,False,140095,121113,Fri Mar 14 18:05:09 +0000 2014,False,Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431,37.403644,43.265905,Fri Mar 14 18:05:09 +0000 2014,0.0618,0.995000,0.0,200.0


# 2. TRAIN MODEL

## 2.1. Political Tweet Prediction

### 2.1.1. Merge dfPolitical data with labels

In [53]:
dfPoliticalAll_train = dfPolitical.merge(trainingTweetDf,
                                         on='tweet_id')

dfPoliticalAll_train.head()

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_url,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,lang_tweet,user_verified,num_political_words,retweet_interactions_ratio,text_len,uppercase_ratio,num_of_political_mentions,digit_exists,isPolitical
0,1585955683513798656,0,None,@AvOzlemZengin YüzüncüYıla YakışanGenelAf adli...,1,0,0,3,2,1564992353168941058,zehra78231638,,0,5,tr,False,1,0.600000,126,0.111111,0,True,Yes
1,1597631718479261696,0,None,#TCYüzyılıÜcretliÖgrtKadro\n#TCYüzyılıÜcretliÖ...,0,2,1,30,28,1324630334416297985,nurozguler,,0,58,tr,False,0,0.517241,140,0.092857,0,False,Yes
2,1572522789948751874,0,None,Ekrem İmamoğlu davayı değerlendirdi. 'Boş işle...,0,0,1,5,66,407597071,onediocom,Türkiye'nin ilk ve tek sosyal içerik sitesi ht...,0,71,tr,True,1,0.070423,131,0.122137,0,False,Yes
3,1591412481561624577,0,None,Sayın Bakanım @suleymansoylu POMEM önlisans er...,1,0,1,0,0,1394789887073738753,buckybarnestr,...,0,0,tr,False,4,0.000000,140,0.078571,1,False,Yes
4,1596914274907348992,0,None,"@varank Sayın bakanım, Bodrumdaki bu araziyi ...",1,0,0,0,0,1586083256088371201,sayariahmet,,0,0,tr,False,2,0.000000,87,0.091954,0,False,Yes


### 2.1.2. Separate X and y values
We only use 3 features here to create a baseline model. However, it is not enough to get good results.

In [54]:
X = dfPoliticalAll_train[['num_political_entities','total_interactions','num_political_words','retweet_interactions_ratio', 'user_verified','num_favorites','is_retweet','text_len','num_mentions', 'num_of_political_mentions','uppercase_ratio', 'digit_exists']]
y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)





### 2.1.3. Train - validation split

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.1.4. Train the model

Here, you may use different models such as neural networks, XGBoost, AdaBoost, RandomForest, Linear Regression, Logistic Regression etc. to see which model does the best. Also, you can use grid_search_cv() or a basic for loop to optimize the hyperparameters of your model.

In [56]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
scoresPol = {}
# create an instance
dtc_politicalDec = DecisionTreeClassifier()

# fit your model
dtc_politicalDec.fit(X_train, y_train)

# make predictions
preds = dtc_politicalDec.predict(X_valid)

# evaluate on validation set
acc_scoreDec = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mseDec = mean_squared_error(y_valid, preds)
scoresPol["DecisionTree"] = mseDec

print("MSE:", mseDec, "\n",
      "Accuracy Score:", acc_scoreDec, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.2916666666666667 
 Accuracy Score: 0.7083333333333334 
 Confusion Matrix: 
 [[119  90]
 [ 85 306]]


In [57]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_politicalRand = RandomForestClassifier()

# fit your model
dtc_politicalRand.fit(X_train, y_train)

# make predictions
preds = dtc_politicalRand.predict(X_valid)

# evaluate on validation set
acc_scoreRand = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mseRand = mean_squared_error(y_valid, preds)
scoresPol["RandForest"] = mseRand

print("MSE:", mseRand, "\n",
      "Accuracy Score:", acc_scoreRand, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.24 
 Accuracy Score: 0.76 
 Confusion Matrix: 
 [[119  90]
 [ 54 337]]


In [58]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

# create an instance
dtc_politicalLog = LogisticRegression()

# fit your model
dtc_politicalLog.fit(X_train, y_train)

# make predictions
preds = dtc_politicalLog.predict(X_valid)

# evaluate on validation set
acc_scoreLog = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mseLog = mean_squared_error(y_valid, preds)
scoresPol["LogisticReg"] = mseLog


print("MSE:", mseLog, "\n",
      "Accuracy Score:", acc_scoreLog, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.23833333333333334 
 Accuracy Score: 0.7616666666666667 
 Confusion Matrix: 
 [[118  91]
 [ 52 339]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [59]:
#Adaboost
from sklearn.ensemble import AdaBoostClassifier

# create an instance
dtc_politicalAda = AdaBoostClassifier()

# fit your model
dtc_politicalAda.fit(X_train, y_train)

# make predictions
preds = dtc_politicalAda.predict(X_valid)

# evaluate on validation set
acc_scoreAda = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mseAda = mean_squared_error(y_valid, preds)
scoresPol["AdaBoost"] = mseAda

print("MSE:", mseAda, "\n",
      "Accuracy Score:", acc_scoreAda, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.21833333333333332 
 Accuracy Score: 0.7816666666666666 
 Confusion Matrix: 
 [[123  86]
 [ 45 346]]


In [60]:
#XGBoost
import xgboost as xgb

# create an instance
dtc_politicalXG = xgb.XGBClassifier()

# fit your model
dtc_politicalXG.fit(X_train, y_train)

# make predictions
preds = dtc_politicalXG.predict(X_valid)

# evaluate on validation set
acc_scoreXG = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mseXG = mean_squared_error(y_valid, preds)
scoresPol["XG"] = mseXG

print("MSE:", mseXG, "\n",
      "Accuracy Score:", acc_scoreXG, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.21666666666666667 
 Accuracy Score: 0.7833333333333333 
 Confusion Matrix: 
 [[115  94]
 [ 36 355]]


In [61]:
import keras
import keras.utils
from tensorflow.keras import utils as np_utils
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from keras.datasets import mnist
from sklearn.utils import shuffle

# create an instance

X_train_nn = np.asarray(X_train).astype('float32')
X_valid_nn = np.asarray(X_valid).astype('float32')
dtc_politicalNNSeq = Sequential()

dtc_politicalNNSeq.add(Dense(1024, activation='sigmoid', input_shape=(X_train_nn.shape[1],), name='hidden_1'))
dtc_politicalNNSeq.add(Dense(1024, activation='relu', name='hidden_2'))
dtc_politicalNNSeq.add(Dense(1024, activation='relu', name='hidden_3'))
dtc_politicalNNSeq.add(Dense(1024, activation='relu', name='hidden_4'))
dtc_politicalNNSeq.add(Dense(1, activation='sigmoid', name='output_layer'))

dtc_politicalNNSeq.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])


# fit your model

dtc_political_ls = []

dtc = dtc_politicalNNSeq.fit(X_train_nn, y_train, batch_size=17, epochs=3, validation_split = 0.2, verbose=1)
dtc_political_ls.append(dtc)

# make predictions

preds = dtc_politicalNNSeq.predict(X_valid_nn)


# evaluate on validation set

acc_scoreNNSeq = accuracy_score(y_valid, preds.round())
confusion = confusion_matrix(y_valid, preds.round())
mseNNSeq = mean_squared_error(y_valid, preds)

scoresPol['NN Sequential'] = mseNNSeq


print("MSE:", mseNNSeq, "\n",
    "Accuracy Score:", acc_scoreNNSeq, "\n",
    "Confusion Matrix:", "\n", confusion)

Epoch 1/3
113/113 [==============================] - 4s 10ms/step - loss: 0.6262 - accuracy: 0.6859 - val_loss: 0.5759 - val_accuracy: 0.6729
Epoch 2/3
113/113 [==============================] - 1s 8ms/step - loss: 0.5597 - accuracy: 0.7188 - val_loss: 0.5401 - val_accuracy: 0.7354
Epoch 3/3
19/19 [==============================] - 0s 4ms/step
MSE: 0.1766996807369624 
 Accuracy Score: 0.7366666666666667 
 Confusion Matrix: 
 [[156  53]
 [105 286]]


In [62]:
bestModelPol = min(scoresPol, key = scoresPol.get)
if bestModelPol == 'DecisionTree':
  dtc_political = dtc_politicalDec
elif bestModelPol == 'RandForest':
  dtc_political = dtc_politicalRand
elif bestModelPol == 'LogisticReg':
  dtc_political = dtc_politicalLog
elif bestModelPol == 'AdaBoost':
    dtc_political = dtc_politicalAda
elif bestModelPol == 'XG':
    dtc_political = dtc_politicalXG
elif bestModelPol == 'NN Sequential':
    dtc_political = dtc_politicalNNSeq

print(bestModelPol)
dtc_political

NN Sequential


## 2.2. Bot Detection

### 2.2.1. Merge dfBotAll data with labels

In [63]:
dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower()

In [64]:
dfBotAll_train = dfBotAll.merge(trainingUserDf,
                               left_on='user_screen_name',
                               right_on='screen_name')

dfBotAll_train

,user_id,user_name,user_screen_name,user_location,user_verified,user_protected,user_favorites_count,user_status_count,user_account_created,user_default_profile,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,status_day_ratio,fav_day_ratio,created_at,difference_in_days_after_created,retweet_total_ratio,num_median_favorites,num_of_tweets,screen_name,isBot
0,1512081815292432394,sezgin,sezgin953116371,,False,False,9627,1535,Thu Apr 07 14:56:21 +0000 2022,True,,46,430,0,0.096639,5.256849,32.969178,Thu Apr 07 14:56:21 +0000 2022,0.6815,0.050251,0.0,199.0,sezgin953116371,No
1,1425452291428077571,Adem Koç,gogoadem61,,False,False,74,113,Wed Aug 11 13:42:03 +0000 2021,False,,14,171,0,0.075676,0.212806,0.139360,Wed Aug 11 13:42:03 +0000 2021,0.2128,0.761062,0.0,113.0,gogoadem61,No
2,328164303,Necmettin Balıkçı,dewil511,,False,False,25,219,Sat Jul 02 21:19:26 +0000 2011,False,,21,49,0,0.300000,0.051847,0.005919,Sat Jul 02 21:19:26 +0000 2011,0.0469,0.010101,0.0,198.0,dewil511,Yes
3,1343666971368431622,Night Bird⁷🦉,midnight__bird,,False,False,84933,15191,Mon Dec 28 21:16:19 +0000 2020,False,"La vie est un sommeil, l’amour en est le rêve...",422,260,48,0.618768,20.067371,112.196830,Mon Dec 28 21:16:19 +0000 2020,0.2642,0.085000,1.0,200.0,midnight__bird,No
4,1240932880488038400,Samed Pınarcı,samedpinarci,,False,False,15783,3734,Fri Mar 20 09:27:20 +0000 2020,False,Orman Mühendisi - Orman İşletme Şefi - Orman G...,133,202,60,0.397015,3.590385,15.175962,Fri Mar 20 09:27:20 +0000 2020,0.1923,0.780000,0.0,200.0,samedpinarci,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1286770207134973954,Hamide Arabacı,anka6054,,False,False,5347,5288,Fri Jul 24 21:08:34 +0000 2020,False,,151,61,0,0.712264,5.785558,5.850109,Fri Jul 24 21:08:34 +0000 2020,0.2188,0.000000,1.0,200.0,anka6054,No
2996,1598032338323214338,atamabekleyenbahceci,atamabekleyenzz,,False,False,196,311,Wed Nov 30 19:13:03 +0000 2022,False,,173,367,0,0.320370,5.654545,3.563636,Wed Nov 30 19:13:03 +0000 2022,3.6364,0.580000,0.0,200.0,atamabekleyenzz,No
2997,760235343966863360,Emrah İNCİ,memrahinci,Istanbul - Bayburt,False,False,725,1029,Mon Aug 01 22:06:45 +0000 2016,False,Researcher | Middle East | Political Science |...,5863,5905,71,0.498215,0.434728,0.306295,Mon Aug 01 22:06:45 +0000 2016,0.0845,0.040000,36.0,200.0,memrahinci,No
2998,1553973684100124672,Murat Kkk,muratkkk18,,False,False,38,18,Mon Aug 01 05:19:56 +0000 2022,False,Normal sıradan bir insanım,1,10,26,0.090909,0.102273,0.215909,Mon Aug 01 05:19:56 +0000 2022,0.0739,0.769231,0.0,13.0,muratkkk18,No


In [65]:
trainingUserDf.isBot.value_counts()

No     2424
Yes     576
Name: isBot, dtype: int64

### 2.2.2. Separate X and y values
We use only 4 features here to create a baseline model. However, it is not enough to get good results.

In [66]:
X = dfBotAll_train[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites','status_day_ratio','fav_day_ratio','user_verified','user_followers_count','user_default_profile','difference_in_days_after_created']]
y = dfBotAll_train.isBot.apply(lambda x: 1 if x=='Yes' else 0)

### 2.2.3. Train-test split

In [67]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.2.4. Train the model

In [68]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

scores = {}
# create an instance
dtc_botDec = DecisionTreeClassifier()

# fit your model
dtc_botDec.fit(X_train, y_train)

# make predictions
preds = dtc_botDec.predict(X_valid)

# evaluate on validation set
acc_scoreDec = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mseDec = mean_squared_error(y_valid, preds)
scores["DecisionTree"] = mseDec

print("MSE:", mseDec, "\n",
      "Accuracy Score:", acc_scoreDec, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.26166666666666666 
 Accuracy Score: 0.7383333333333333 
 Confusion Matrix: 
 [[406  87]
 [ 70  37]]


In [69]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_botRand = RandomForestClassifier()

# fit your model
dtc_botRand.fit(X_train, y_train)

# make predictions
preds = dtc_botRand.predict(X_valid)

# evaluate on validation set
acc_scoreRand = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mseRand = mean_squared_error(y_valid, preds)
scores["RandForest"] = mseRand

print("MSE:", mseRand, "\n",
      "Accuracy Score:", acc_scoreRand, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.155 
 Accuracy Score: 0.845 
 Confusion Matrix: 
 [[475  18]
 [ 75  32]]


In [70]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

# create an instance
dtc_botLog = LogisticRegression()

# fit your model
dtc_botLog.fit(X_train, y_train)

# make predictions
preds = dtc_botLog.predict(X_valid)

# evaluate on validation set
acc_scoreLog = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mseLog = mean_squared_error(y_valid, preds)
scores["LogisticReg"] = mseLog

print("MSE:", mseLog, "\n",
      "Accuracy Score:", acc_scoreLog, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.19 
 Accuracy Score: 0.81 
 Confusion Matrix: 
 [[449  44]
 [ 70  37]]


In [71]:
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier
# create an instance
dtc_botAda = AdaBoostClassifier()

# fit your model
dtc_botAda.fit(X_train, y_train)

# make predictions
preds = dtc_botAda.predict(X_valid)

# evaluate on validation set
acc_scoreAda = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mseAda = mean_squared_error(y_valid, preds)
scores["AdaBoost"] = mseAda

print("MSE:", mseAda, "\n",
      "Accuracy Score:", acc_scoreAda, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.15833333333333333 
 Accuracy Score: 0.8416666666666667 
 Confusion Matrix: 
 [[470  23]
 [ 72  35]]


In [72]:
#XGBoost
import xgboost as xgb

# create an instance
dtc_botXG = xgb.XGBClassifier()

# fit your model
dtc_botXG.fit(X_train, y_train)

# make predictions
preds = dtc_botXG.predict(X_valid)

# evaluate on validation set
acc_scoreXG = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mseXG = mean_squared_error(y_valid, preds)
scores["XG"] = mseXG

print("MSE:", mseXG, "\n",
      "Accuracy Score:", acc_scoreXG, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.16166666666666665 
 Accuracy Score: 0.8383333333333334 
 Confusion Matrix: 
 [[469  24]
 [ 73  34]]


In [73]:
import keras
import keras.utils
from tensorflow.keras import utils as np_utils
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from keras.datasets import mnist
from sklearn.utils import shuffle

# create an instance

X_train_nn = np.asarray(X_train).astype('float32')
X_valid_nn = np.asarray(X_valid).astype('float32')
dtc_botNNSeq = Sequential()

dtc_botNNSeq.add(Dense(1024, activation='sigmoid', input_shape=(X_train_nn.shape[1],), name='hidden_1'))
dtc_botNNSeq.add(Dense(1024, activation='relu', name='hidden_2'))
dtc_botNNSeq.add(Dense(1024, activation='relu', name='hidden_3'))
dtc_botNNSeq.add(Dense(1024, activation='relu', name='hidden_4'))
dtc_botNNSeq.add(Dense(1, activation='sigmoid', name='output_layer'))

dtc_botNNSeq.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])


# fit your model

dtc_political_ls = []

dtc = dtc_botNNSeq.fit(X_train_nn, y_train, batch_size=17, epochs=3, validation_split = 0.2, verbose=1)
dtc_political_ls.append(dtc)

# make predictions

preds = dtc_botNNSeq.predict(X_valid_nn)


# evaluate on validation set

acc_scoreNNSeq = accuracy_score(y_valid, preds.round())
confusion = confusion_matrix(y_valid, preds.round())
mseNNSeq = mean_squared_error(y_valid, preds)

scores['NN Sequential'] = mseNNSeq


print("MSE:", mseNNSeq, "\n",
    "Accuracy Score:", acc_scoreNNSeq, "\n",
    "Confusion Matrix:", "\n", confusion)

Epoch 1/3
113/113 [==============================] - 1s 5ms/step - loss: 0.5236 - accuracy: 0.8099 - val_loss: 0.5092 - val_accuracy: 0.8083
Epoch 2/3
113/113 [==============================] - 0s 4ms/step - loss: 0.4524 - accuracy: 0.8219 - val_loss: 0.4803 - val_accuracy: 0.7917
Epoch 3/3
19/19 [==============================] - 0s 2ms/step
MSE: 0.1264066327594548 
 Accuracy Score: 0.8366666666666667 
 Confusion Matrix: 
 [[484   9]
 [ 89  18]]


In [74]:
bestModelBot = min(scores, key = scores.get)
if bestModelBot == 'DecisionTree':
  dtc_bot = dtc_botDec
elif bestModelBot == 'RandForest':
  dtc_bot = dtc_botRand
elif bestModelBot == 'LogisticReg':
  dtc_bot = dtc_botLog
elif bestModelBot == 'AdaBoost':
  dtc_bot = dtc_botAda
elif bestModelBot == 'XG':
  dtc_bot = dtc_botXG
elif bestModelBot == 'NN Sequential':
  dtc_bot = dtc_botNNSeq

print(bestModelBot)
dtc_bot


NN Sequential


# 3. MAKE PREDICTIONS

Here, you will make predictions with the models that you have trained above.

## 3.1. Predictions for Tweets (Political or Not)

In [75]:
# read the evaluation file as follows
evaluationTweetDf = pd.read_csv('/content/drive/My Drive/CS412_project/raw_data/evaluation-round{}-tweet.csv'.format(ROUND), dtype={0: str}, header=None, names=['tweet_id'])
evaluationTweetDf = evaluationTweetDf.dropna()
evaluationTweetDf


# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfPolitical_test[['num_political_entities','total_interactions','num_political_words','retweet_interactions_ratio', 'user_verified','num_favorites','is_retweet','text_len','num_mentions', 'num_of_political_mentions','uppercase_ratio', 'digit_exists']]

X = np.asarray(X.astype('float32'))

# make predictions based on these variables
predictions_political = dtc_political.predict(X)

157/157 [==============================] - 0s 2ms/step


### This part is important! We expect you to return your predictions in the following format:

In [76]:
modelPredTweet = dict([(x,float(y)) for x,y in zip([*dfPolitical_test.tweet_id], predictions_political)])
modelPredTweet

{'1434787703783051264': 0.33561980724334717,
 '1367571642604544000': 0.26128923892974854,
 '1589993032975544320': 0.8858343958854675,
 '1565312596135354373': 0.8259439468383789,
 '1579558096833511424': 0.9453140497207642,
 '1439547067337256967': 0.18371295928955078,
 '1559963768372740098': 0.7173845767974854,
 '1562853131251118081': 0.4170932173728943,
 '1586021183958704128': 0.7749304175376892,
 '1585766233491886081': 0.20029516518115997,
 '1427746815420604417': 0.44941723346710205,
 '1352635736537882629': 0.3881653845310211,
 '1415032260571680768': 0.15881629288196564,
 '1548636597628899328': 0.8629420399665833,
 '1564926450096013313': 0.5834999680519104,
 '1585634359612420101': 0.8986257910728455,
 '1597138789108895744': 0.6145207285881042,
 '1391681495622995971': 0.1239282488822937,
 '1389951943343316995': 0.1679374873638153,
 '1452348722810138646': 0.6383229494094849,
 '1595829502021623812': 0.6931070685386658,
 '1413108476348354562': 0.3384627401828766,
 '1579408398894137344': 0.

## 3.2. Predictions for Users (Bot or Not)

In [77]:
evaluationUserDf = pd.read_csv('/content/drive/My Drive/CS412_project/raw_data/evaluation-round{}-user.csv'.format(ROUND), dtype={0: str}, header=None, names=['user_screen_name'])
evaluationUserDf = evaluationUserDf.dropna()
# merge it with the political dataframe so that you can use the make predictions based on the variables
dfBot_test = dfBotAll.merge(evaluationUserDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfBot_test[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites','status_day_ratio','fav_day_ratio','user_verified','user_followers_count','user_default_profile','difference_in_days_after_created']]
X = np.asarray(X.astype('float32'))
# make predictions based on these variables
predictions_bot = dtc_bot.predict(X)

157/157 [==============================] - 0s 1ms/step


In [78]:
modelPredUser = dict([(x,float(y)) for x,y in zip([*dfBot_test.user_screen_name], predictions_bot)])
modelPredUser

{'biologselim': 0.13937751948833466,
 'omerakdag34': 0.13287818431854248,
 'bilgin21604923': 0.13995477557182312,
 '_sydneycarton_': 0.0838116854429245,
 'denizlihabercom': 0.140767440199852,
 'burakerbaychp': 0.139715775847435,
 'mvnez': 0.22119919955730438,
 'qara118': 0.14279572665691376,
 'nabiyonyevrum': 0.20345619320869446,
 'farukhalit2': 0.21717247366905212,
 'harlunoshi': 0.15574774146080017,
 'heritagepaix': 0.21343538165092468,
 'nuranwolf': 0.14129133522510529,
 'politikgundem': 0.14481469988822937,
 'isakethudax': 0.14156180620193481,
 'enveraysevera': 0.14288395643234253,
 'ilaydejaneiro': 0.2191038727760315,
 '1905anason': 0.1399163454771042,
 'eraydurgut03': 0.09087339043617249,
 'dasiskein': 0.14285719394683838,
 'ercan_bas29': 0.1304742395877838,
 'mett_1907': 0.13463173806667328,
 'ondemir066': 0.13967347145080566,
 'semihyeteer': 0.12670810520648956,
 'haberinyokcokk': 0.1633126437664032,
 'meleky_ozaydin': 0.14654378592967987,
 'mehmetaltay64': 0.13842979073524475,

# PREPARE SUBMISSION

You will need to submit exact same file produced by using the following code. Any deviation from the desired format willbe marked as 0.

In [80]:
# Explain your approach

data_explanations = '''
Although we have achieved a high accuracy score from the annotations, we thought that using the data that Onur Hoca provided would be more accurate 
and supply better outcomes to train our model.For that reason, we analyzed the raw data such as tweet_metadata.json and training-tweet.csv.
 With analyzing the features in the tweet_metadata.json, we became able to observe the qualities of the tweets such as its hashtags, mentions and 
 the length of the text in the tweet.After searching and thinking about the features that might lead a higher accuracy to understand bot accounts and 
 political tweets, we came up with several feature opinions. For instance, we looked for specific words such as "parti" in the tweets to understand if 
 that tweet belongs to a political tweet type.
'''

feature_explanations = '''

We thought that the number of tweets that a user shares per day would be an indicator of a bot account.
For that reason, we found how many days have passed since the user created his/her account. After that, we found the total number 
of tweets that this user posted. Dividing the total number of tweets of that user to how many days has passed since the user created the account,
we became able to observe how many tweets a user post per day. We used this feature to train our model. 
Then we also created a feature that calculates the statuses(tweets + retweets )shared per day.
Thridly we implemented  another manually crafted that calculates the ratio of posts liked per day. Because we thought a high ratio of  posts liked/day
can indicate that an account is a bot.
We also added features for Bot Detection from raw data that includes user_verified

We created two manually crafted features for  Political Tweet Prediction, first one is a feature that checks if  the tweet text contain cetain political words and
counts them. We included words that can be seen in majority of political tweets. Second one was a feature thatcalculates retweet/ total interactions ratio. We thought that 
this ratio could point out a political tweet since many political tweets have great amounts of retweets.
We also added features from raw data that includes user_verified,user_protected,user_default_profile,created_at
Also implemented get_uppercase_ratio, num_url, digit_exists for tweet.

'''

model_explanations = '''
We tried multiple models to see which one results the best for the data. For that purpose,
we used Random Forest, Decision Tree, Logistic Regression, AdaBoost and XGBoost, NN Sequential for both political tweet
detection and bot user detection. We fit our models to training data and made predictions. Then, we calculated
their MSE scores by evaluating them on the validation set. According to these calculations, the best models were:
-For political tweet detection: NN Sequential, MSE: 0.1644207879352009
-For bot account detection: NN Sequential, MSE: 0.12283942350398007 
'''

additional_explanations = '''
No additional explanation.
'''


In [81]:

STUDENT_ID = '28195'#'<insert-your-id-here>'
PROJECT_CODE = 'CS412da78f316bc56'

predictions = {
    'round': ROUND,
    'student_id': STUDENT_ID,
    'user_predictions': modelPredUser,
    'tweet_predictions': modelPredTweet,
    'explanations': {
        'data': data_explanations,
        'feature': feature_explanations,
        'model': model_explanations,
        'other': additional_explanations,
    }
}


with open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'w') as fl:
    fl.write(json.dumps(predictions, indent=4))

In [82]:
# Test your submission file

submission = json.load(open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'r'))
submission

{'round': 3,
 'student_id': '28195',
 'user_predictions': {'biologselim': 0.13937751948833466,
  'omerakdag34': 0.13287818431854248,
  'bilgin21604923': 0.13995477557182312,
  '_sydneycarton_': 0.0838116854429245,
  'denizlihabercom': 0.140767440199852,
  'burakerbaychp': 0.139715775847435,
  'mvnez': 0.22119919955730438,
  'qara118': 0.14279572665691376,
  'nabiyonyevrum': 0.20345619320869446,
  'farukhalit2': 0.21717247366905212,
  'harlunoshi': 0.15574774146080017,
  'heritagepaix': 0.21343538165092468,
  'nuranwolf': 0.14129133522510529,
  'politikgundem': 0.14481469988822937,
  'isakethudax': 0.14156180620193481,
  'enveraysevera': 0.14288395643234253,
  'ilaydejaneiro': 0.2191038727760315,
  '1905anason': 0.1399163454771042,
  'eraydurgut03': 0.09087339043617249,
  'dasiskein': 0.14285719394683838,
  'ercan_bas29': 0.1304742395877838,
  'mett_1907': 0.13463173806667328,
  'ondemir066': 0.13967347145080566,
  'semihyeteer': 0.12670810520648956,
  'haberinyokcokk': 0.16331264376640